In [527]:
import pandas as pd
import numpy as np
import scipy as cp
import sys

In [528]:
def data_split(s1,s2,data):
    return np.vstack([data[:s1],data[s2:]]) ,data[s1:s2]

In [529]:
def Accuracy(Xva,Yva, mu1, mu2, shared_sigma, N1, N2):
    result = predict(Xva, mu1, mu2, shared_sigma, N1, N2)
    correct = 0
    for i in range(len(Xva)):
        if result[i] >= 0.5:
            if Yva[i] ==1:
                correct = correct+1
        else:
            if Yva[i] ==0:
                correct = correct+1
    accuracy = correct/len(Yva)   
    return accuracy

In [530]:
def fscaling(data):
    N = data.T
    f =  1/(np.max(N[0])-np.min(N[0]))
    for i in range(len(data)):
        data[i,0]= (data[i,0]-np.min(N[0]))*f
        
    f =  1/(np.max(N[1])-np.min(N[1]))
    for i in range(len(data)):
        data[i,1]= (data[i,1]-np.min(N[1]))*f
    
    f =  1/(np.max(N[3])-np.min(N[3]))
    for i in range(len(data)):
        data[i,3]= (data[i,3]-np.min(N[3]))*f
    
    f =  1/(np.max(N[4])-np.min(N[4]))
    for i in range(len(data)):
        data[i,4]= (data[i,4]-np.min(N[4]))*f
    
    f =  1/(np.max(N[5])-np.min(N[5]))
    for i in range(len(data)):
        data[i,5]= (data[i,5]-np.min(N[5]))*f
    return data

In [531]:
def shuffle(X,Y):
    rand = np.arange(len(X))
    np.random.shuffle(rand)
    return (X[rand],Y[rand])

In [532]:
def sigmoid(z):
    res = 1 / (1.0 + np.exp(-z))
    return np.clip(res, 0.00000000000001, 0.99999999999999)

In [533]:
def feature_filter(Xtr,col):
    return np.delete(Xtr,col,axis = 1)

In [534]:
def predict(Xte, mu1, mu2, shared_sigma, N1, N2):
    sigma_inverse = np.linalg.inv(shared_sigma)
    w = np.dot( (mu1-mu2), sigma_inverse)
    x = Xte.T
    b = (-0.5) * np.dot(np.dot([mu1], sigma_inverse), mu1) + (0.5) * np.dot(np.dot([mu2], sigma_inverse), mu2) + np.log(float(N1)/N2)
    a = np.dot(w, x) + b
    y = sigmoid(a)
    return y

In [535]:
def train(Xtr,Ytr):
    datalength = len(Xtr)
    fnumber = len(Xtr[0])
    
    cnt1 = 0
    cnt2 = 0
    mu1 = np.zeros((fnumber,))
    mu2 = np.zeros((fnumber,))
    for i in range(datalength):
        if Ytr[i] == 1:
            mu1 += Xtr[i]
            cnt1 += 1
        else:
            mu2 += Xtr[i]
            cnt2 += 1
    mu1 /= cnt1
    mu2 /= cnt2
    
    sigma1 = np.zeros((fnumber,fnumber))
    sigma2 = np.zeros((fnumber,fnumber))
    for i in range(datalength):
        if Ytr[i] == 1:
            sigma1 += np.dot(np.transpose([Xtr[i] - mu1]), [(Xtr[i] - mu1)])
        else:
            sigma2 += np.dot(np.transpose([Xtr[i] - mu2]), [(Xtr[i] - mu2)])
    sigma1 /= cnt1
    sigma2 /= cnt2
    shared_sigma = (float(cnt1) / datalength) * sigma1 + (float(cnt2) / datalength) * sigma2
    
    return mu1, mu2, shared_sigma, cnt1, cnt2

In [536]:
s1 = 2000
s2 = 8500
col = [i for i in range(32,38)]  #drop marry situation

In [537]:
X = pd.read_csv('X_train.csv',encoding = 'big5')
Y = pd.read_csv('Y_train.csv',encoding = 'big5',names = ['haha'])
X = np.array(X,float)
Y = np.array(Y,float)
#[X,Y] = shuffle(X,Y)
[Xtr,Xva] = data_split(s1,s2,X)
[Ytr,Yva] = data_split(s1,s2,Y)
Ytr = np.reshape(Ytr,(len(Ytr)))
Yva = np.reshape(Yva,(len(Yva)))

In [538]:
Xtr = fscaling(Xtr)
Xtr = feature_filter(Xtr,col)

Xva = fscaling(Xva)
Xva = feature_filter(Xva,col)

In [539]:
[mu1, mu2, shared_sigma, N1, N2] = train(Xtr,Ytr)

In [540]:
Accuracy(Xtr,Ytr, mu1, mu2, shared_sigma, N1, N2)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


0.7577990100149649

In [541]:
Accuracy(Xva,Yva, mu1, mu2, shared_sigma, N1, N2)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


0.7647692307692308

In [542]:
Xte = pd.read_csv('X_test.csv',encoding ='big5')
Xte = np.array(Xte,float)
Xte = fscaling(Xte)
Xte = feature_filter(Xte,col)

In [543]:
result = predict(Xte, mu1, mu2, shared_sigma, N1, N2)
Result = np.zeros(len(Xte),int)
for i in range(len(Xte)):
        if result[i] >= 0.5:
            Result[i]=1      
        else:
            Result[i]=0  

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


In [544]:
final = []
num=1
while num <= len(Xte):
    final.append([str(num),Result[num-1]])
    num = num+1

In [545]:
Final = pd.DataFrame(final,columns = ['id','label'] )
print(Final)
Final.to_csv("hw2_generative.csv",index = False)

          id  label
0          1      0
1          2      0
2          3      0
3          4      0
4          5      0
5          6      0
6          7      0
7          8      0
8          9      0
9         10      0
10        11      0
11        12      0
12        13      0
13        14      0
14        15      0
15        16      0
16        17      0
17        18      0
18        19      0
19        20      0
20        21      0
21        22      0
22        23      0
23        24      0
24        25      0
25        26      0
26        27      0
27        28      0
28        29      0
29        30      0
...      ...    ...
16251  16252      0
16252  16253      0
16253  16254      0
16254  16255      0
16255  16256      0
16256  16257      0
16257  16258      0
16258  16259      0
16259  16260      0
16260  16261      0
16261  16262      0
16262  16263      0
16263  16264      0
16264  16265      0
16265  16266      0
16266  16267      0
16267  16268      0
16268  16269      0
